In [1]:
import numpy as np
import pickle

from tqdm import tqdm_notebook as tqdm
from spacy.lang.xx import MultiLanguage
nlp = MultiLanguage()

from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, load_model
from keras_multi_head import MultiHeadAttention
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_accuracy

Using TensorFlow backend.


In [2]:
## Path of file
filename_test = "../data/test/clear-sent-comb-with-label.txt" 

In [3]:
# Load pickle file
word2idx = pickle.load(open("../data/pickle_file/word2idx.pkl","rb"))
char2idx = pickle.load(open("../data/pickle_file/char2idx.pkl","rb"))
label2idx = pickle.load(open("../data/pickle_file/label2idx.pkl","rb"))
idx2label = {v:k for k,v in label2idx.items()}

maxwordlength = 15

# Load Model

In [4]:
model = load_model("../model/ner_order_v5.h5",
                   custom_objects ={'CRF':CRF,
                                   'crf_loss':crf_loss,
                                   'crf_accuracy':crf_accuracy,
                                   'MultiHeadAttention':MultiHeadAttention})

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [5]:
## Predict data
def predict(dataset):
    predLabels = []
    
    for i, data in enumerate(dataset):
        tokens, char= data
        tokens = np.asarray([tokens])
        char = np.asarray([char])
        
        pred = model.predict([tokens, char], verbose = False)[0]
        predLabels.append([np.argmax(i) for i in pred])
        
    return predLabels

In [6]:
# Calculate accuracy model for each word/entity
def checkaccuracy_word(predict_label,correct_label):
    counter = 0
    idx_wrong_pred = []
    
    for idx_sentence in tqdm(range(len(predict_label))):
        for idx_word in range(len(predict_label[idx_sentence])):
            if (predict_label[idx_sentence][idx_word]) == (correct_label[idx_sentence][idx_word]):
                counter += 1
            else :
                idx_wrong_pred.append(idx_sentence)
    return counter, idx_wrong_pred

In [7]:
# Check entity of word which is wrong prediction
def checkaccuracy_entity(predict_label, correct_label):
    correct_entity = []
    wrong_entity = []
    
    for idx_sentence in tqdm(range(len(predict_label))):
        for idx_word in range(len(predict_label[idx_sentence])):
            if (predict_label[idx_sentence][idx_word]) == (correct_label[idx_sentence][idx_word]):
                correct_entity.append(predict_label[idx_sentence][idx_word])
            else :
                wrong_entity.append(correct_label[idx_sentence][idx_word])
    return correct_entity, wrong_entity

In [8]:
# Calculate accuracy model for each sentence/order
def checkaccuracy_sent(predict_label, correct_label):
    count = 0
    for idx in range(len(predict_label)):
        if predict_label[idx] == correct_label[idx] :
            count += 1
    return count

## Testing

In [40]:
def convert_to_vector(text, word2Idx, char2Idx):
    """
        Args:
            text (str): Input sentence
        Return:
            A list consist of.
            [0]: A list of Token's index in the sentence
            [1]: A list of Casing condition of token in the sentence
            [2]: A list of list of token's Characters pattern
    """
    word_indices = []
    char_indices = []
    doc = nlp(text)
    for token in doc:
        i = token.text
        try:
            word_indices.append(word2Idx[i])
            unknown = False
        except KeyError:
            unknown = True #word_indices.append(word2Idx["UNK"])
        
        if unknown == False:
            tok = []
            for j in i:
                try:
                    tok.append(char2Idx[j])
                except KeyError:
                    tok.append(char2Idx["UNK"])
            char_indices.append(pad_sequences([tok], maxlen=maxwordlength)[0])

    return [word_indices, char_indices]

In [10]:
# Split testing data
data = []
sentences = []
labels = []

with open(filename_test, encoding='utf-8') as f:
    for line in f:
        data.append(line.rstrip())
        sentences.append(line.rstrip().split(':')[0].lower())
        labels.append(line.rstrip().split(':')[1])

In [11]:
# Do compute prediction on testing data
predLabels = []

for line in tqdm(sentences):
    tmp = convert_to_vector(line, word2idx, char2idx)
    ans = predict([tmp])[0]
    ans = [idx2label[i] for i in ans]
    predLabels.append(ans)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


## Evaluate Performance

In [12]:
correctLabels = []

for i in labels:
    correctLabels.append(i.split())

In [13]:
# Performance on tes dataset
total_word = 0
for i in predLabels:
    total_word += len(i)

# Calculate Performance of model on data
counter, idx_wrong_pred = checkaccuracy_word(predLabels, correctLabels)
correct_entity, wrong_entity = checkaccuracy_entity(predLabels, correctLabels)
count_sent = checkaccuracy_sent(predLabels, correctLabels)

print("Accuracy per sentence :", count_sent*100/len(predLabels))
print("Accuracy per word :", counter*100/total_word)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """



Accuracy per sentence : 62.505295529788334
Accuracy per word : 90.46048642635328


In [14]:
from collections import Counter
total_wrong_entity = Counter(wrong_entity)
name_wrong_entity = set(wrong_entity)

for i in name_wrong_entity: 
    print (i+' = ', round(total_wrong_entity[i]/len(wrong_entity),2))

desc =  0.33
numeric =  0.01
o-brand =  0.14
b-brand =  0.38
uom =  0.05
i-brand =  0.02
color =  0.02
brand =  0.0
flavor =  0.02
item =  0.03


## Testing use custom data

In [47]:
input_data = 'gw beli abese kopi tai susu tiga saset lol emang oke'

In [48]:
# Do compute prediction on testing data
predLabels = []

tmp = convert_to_vector(input_data, word2idx, char2idx)
ans = predict([tmp])[0]
ans = [idx2label[i] for i in ans]
predLabels.append(ans)

In [49]:
predLabels

[['soo', 'brand', 'flavor', 'flavor', 'numeric', 'uom', 'eoo']]